<a href="https://colab.research.google.com/github/profteachkids/chetools/blob/main/tools/IsopropanolSynthesis_che2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/che2.ipynb -O che2.ipynb
!pip install importnb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from importnb import Notebook
with Notebook(): 
    from che2 import DotDict, Range, RangeArray, Comp, CompArray, d2nt, Props

In [3]:
import jax
import jax.numpy as jnp
from scipy.optimize import least_squares

<img src='https://github.com/profteachkids/CHE2064_Spring2022/raw/main/IsopropylSynthesisProcessDiagram.jpg'>

In [4]:
# Static parameters (Total feed, feed mole fradtions, feed temperature and )
# 1. propene
# 2. diisopropyl ether
# 3. isopropanol
# 4. water
# Rx1:  C3H6 + H2O -> C3H8O
# Rx2:  2C3H8O -> C6H14 + H2O 

#Simple version - Overall mass balance only

e = DotDict()
e.alkeneF = 100.
e.waterF = 100.

e.alkeney = jnp.array([1., 0., 0., 0.])
e.waterx = jnp.array([0., 0., 0., 1.])

e.rx1 = jnp.array([-1, 0, 1, -1])
e.rx2 = jnp.array([0, 1, -2, 1])

e.rx1_extent = Range(e.alkeneF/2, 0., e.alkeneF)
e.rx2_extent = Range(e.alkeneF/4, 0., e.alkeneF/2)

e.alcoholP = Range(e.alkeneF/2, 0., e.alkeneF)  
e.alcoholPx = jnp.array([0., 0.005, 0.94, 0.055])

e.etherP = Range(e.alkeneF/2, 0., e.alkeneF)
e.etherPx = jnp.array([0.001, 0.98, 0.018, 0.002])

In [5]:
x, x2nt, wrap, nt0, x2unk, x2const  = d2nt(e)

In [6]:
def eqs(d):
    r=DotDict()
    feed = d.alkeneF*d.alkeney + d.waterF*d.waterx 
    product = (d.etherP*d.etherPx + d.alcoholP*d.alcoholPx)

    molecular_balance = feed + d.rx1_extent*d.rx1 + d.rx2_extent*d.rx2 - product

    r.alkene = d.alkeneF*d.alkeney
    r.water = d.waterF*d.waterx 
    r.ether = d.etherP*d.etherPx
    r.alcohol = d.alcoholP*d.alcoholPx
    return molecular_balance, r

In [7]:
eqs_wrapped, jac = wrap(eqs)
eqs_wrapped(x)

DeviceArray([ 49.95, -24.25, -47.9 ,  72.15], dtype=float64)

In [8]:
sol=least_squares(eqs_wrapped, x, jac=jac, x_scale='jac')
print(sol)

 active_mask: array([0., 0., 0., 0.])
        cost: 3.671435684721282e-20
         fun: array([ 1.56442134e-10,  1.77635684e-15, -1.56461510e-10,  1.56442859e-10])
        grad: array([-2.28048451e-12,  2.20084537e-09,  6.40141415e-10,  1.08419890e-11])
         jac: array([[-4.85885054e-03,  0.00000000e+00,  0.00000000e+00,
        -4.62297928e-03],
       [ 0.00000000e+00,  4.68895830e+00, -2.31148964e-02,
        -4.53051969e+00],
       [ 4.85885054e-03, -9.37791660e+00, -4.34560052e+00,
        -8.32136270e-02],
       [-4.85885054e-03,  4.68895830e+00, -2.54263860e-01,
        -9.24595855e-03]])
     message: '`gtol` termination condition is satisfied.'
        nfev: 19
        njev: 19
  optimality: 2.200845365517745e-09
      status: 1
     success: True
           x: array([ 9.93202639, -2.14550313,  2.97450864, -2.97450864])


In [9]:
nt=x2nt(sol.x)
nt._asdict()

OrderedDict([('alkeneF', DeviceArray(100., dtype=float64)),
             ('waterF', DeviceArray(100., dtype=float64)),
             ('alkeney', DeviceArray([1., 0., 0., 0.], dtype=float64)),
             ('waterx', DeviceArray([0., 0., 0., 1.], dtype=float64)),
             ('rx1', DeviceArray([-1,  0,  1, -1], dtype=int64)),
             ('rx2', DeviceArray([ 0,  1, -2,  1], dtype=int64)),
             ('rx1_extent', DeviceArray(99.99514091, dtype=float64)),
             ('rx2_extent', DeviceArray(5.23760933, dtype=float64)),
             ('alcoholP', DeviceArray(95.14091351, dtype=float64)),
             ('alcoholPx',
              DeviceArray([0.   , 0.005, 0.94 , 0.055], dtype=float64)),
             ('etherP', DeviceArray(4.85908649, dtype=float64)),
             ('etherPx',
              DeviceArray([0.001, 0.98 , 0.018, 0.002], dtype=float64))])

In [10]:
unk=x2unk(sol.x)
unk._asdict()

OrderedDict([('rx1_extent', DeviceArray(99.99514091, dtype=float64)),
             ('rx2_extent', DeviceArray(5.23760933, dtype=float64)),
             ('alcoholP', DeviceArray(95.14091351, dtype=float64)),
             ('etherP', DeviceArray(4.85908649, dtype=float64))])

In [11]:
x2const()._asdict()

OrderedDict([('alkeneF', DeviceArray(100., dtype=float64)),
             ('waterF', DeviceArray(100., dtype=float64)),
             ('alkeney', DeviceArray([1., 0., 0., 0.], dtype=float64)),
             ('waterx', DeviceArray([0., 0., 0., 1.], dtype=float64)),
             ('rx1', DeviceArray([-1,  0,  1, -1], dtype=int64)),
             ('rx2', DeviceArray([ 0,  1, -2,  1], dtype=int64)),
             ('alcoholPx',
              DeviceArray([0.   , 0.005, 0.94 , 0.055], dtype=float64)),
             ('etherPx',
              DeviceArray([0.001, 0.98 , 0.018, 0.002], dtype=float64))])

In [12]:
_, r= eqs(x2nt(sol.x))
r

{'alkene': DeviceArray([100.,   0.,   0.,   0.], dtype=float64),
 'water': DeviceArray([  0.,   0.,   0., 100.], dtype=float64),
 'ether': DeviceArray([0.00485909, 4.76190476, 0.08746356, 0.00971817], dtype=float64),
 'alcohol': DeviceArray([ 0.        ,  0.47570457, 89.4324587 ,  5.23275024], dtype=float64)}

<img src='https://github.com/profteachkids/CHE2064_Spring2022/raw/main/IsopropylSynthesisProcessDiagram.jpg'>

In [13]:
# Include internal flows (recycle)


d=DotDict()
d.alkeneF = 100.
d.waterF = 100.

d.alkeney = jnp.array([1., 0., 0., 0.])
d.waterx = jnp.array([0., 0., 0., 1.])

d.rx1 = jnp.array([-1, 0, 1, -1])
d.rx2 = jnp.array([0, 1, -2, 1])

d.rx1_extent = Range(d.alkeneF/2, 0., d.alkeneF)
d.rx2_extent = Range(d.alkeneF/4, 0., d.alkeneF/2)

d.alcoholP = Range(d.alkeneF/2, 0., d.alkeneF)  
d.alcoholPx = jnp.array([0., 0.005, 0.94, 0.055])

d.etherP = Range(d.alkeneF/2, 0., d.alkeneF)
d.etherPx = jnp.array([0.001, 0.98, 0.018, 0.002])

d.WARFR = 12. #water_alkene_reactor_feed_ratio

d.D1F = Range(d.alkeneF * d.WARFR, 0., 2.*d.WARFR*d.alkeneF)
d.D1Fx = Comp([0.001, 0.03, 0.07, 0.9])

d.D1K = jnp.array([100., 10., 5., 0.2])
d.D1Vy = Comp([0.01, 0.299, 0.3, 0.3])
d.D1Lx = Comp([0.0001, 0.1, 0.1, 0.8])
d.D1V = Range(d.alkeneF * d.WARFR/2, 0., 2.*d.WARFR*d.alkeneF)
d.D1L = Range(d.alkeneF * d.WARFR/2, 0., 2.*d.WARFR*d.alkeneF)

d.decantW = Range(d.alkeneF * d.WARFR, 0., 2.*d.WARFR*d.alkeneF)
d.decantWx = jnp.array([0.001, 0.005, 0.02, 0.98]) 


In [14]:
x, x2nt, wrap, nt0, x2unk, x2const  = d2nt(d)

In [15]:
def eqs2(d):
    r=DotDict()
    r.reactor_feed = d.alkeneF*d.alkeney + d.waterF*d.waterx + d.D1L*d.D1Lx + d.decantW*d.decantWx

    r.REACTOR = r.reactor_feed + d.rx1_extent*d.rx1 + d.rx2_extent*d.rx2 - d.D1F*d.D1Fx

    r.FEED_RATIO = jnp.atleast_1d((d.waterF+d.D1L+d.decantW)/d.alkeneF - d.WARFR)
    
    r.D1 = d.D1F*d.D1Fx - d.D1V*d.D1Vy - d.D1L*d.D1Lx
    r.D1K = d.D1Vy/d.D1Lx - d.D1K

    r.AZ = d.D1V*d.D1Vy - d.etherP*d.etherPx - d.alcoholP*d.alcoholPx - d.decantW*d.decantWx

    r.alkene = d.alkeneF*d.alkeney
    r.water = d.waterF*d.waterx 
    r.ether = d.etherP*d.etherPx
    r.alcohol = d.alcoholP*d.alcoholPx
    return jnp.concatenate([r.REACTOR, r.FEED_RATIO, r.D1, r.D1K, r.AZ]), r


In [16]:
eqs2_wrapped, jac = wrap(eqs2)
eqs2_wrapped(x)

DeviceArray([ 5.00611928e+01,  5.50299646e+01,  7.79166839e-02,
              6.52030926e+02,  7.00000000e+00, -5.46185287e+00,
             -2.21389701e+02, -1.74097719e+02,  4.00949272e+02,
              1.00220022e+01, -6.71034213e+00, -1.69933993e+00,
              2.12582508e-01,  5.35066007e+00,  1.42109736e+02,
              1.26119802e+02, -9.80830198e+02], dtype=float64)

In [17]:
sol=least_squares(jax.jit(eqs2_wrapped),x,jac=jac, x_scale='jac')
sol

 active_mask: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
        cost: 5.355068049086058e-26
         fun: array([ 1.66121067e-14, -3.85561622e-16,  8.53676808e-14, -2.00371981e-13,
        0.00000000e+00, -6.72205347e-18,  1.77635684e-15, -8.52651283e-14,
        2.27373675e-13,  1.42108547e-14,  1.77635684e-15, -8.88178420e-16,
       -1.11022302e-16, -5.20417043e-18, -1.66533454e-15, -2.44249065e-15,
       -1.42108547e-14])
        grad: array([ 1.30765053e-15, -1.74191474e-12,  1.42658895e-14,  7.87949645e-15,
        1.76526332e-10, -6.48744411e-15, -3.02326860e-12, -1.21061920e-10,
        1.42038086e-12, -1.96757757e-13,  4.11345709e-12, -1.42045228e-12,
        1.65103418e-12,  8.59496457e-11,  2.65788164e-12, -1.97237510e-10,
       -2.55044428e-12])
         jac: array([[-4.85885038e-03,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -1.04349163e-02, -2.08710361e-02,
         1.80035965e-04,  4.52937879e-03,  0.00000000e+00,
     

In [18]:
unk=x2unk(sol.x)
unk._asdict()

OrderedDict([('rx1_extent', DeviceArray(99.99514091, dtype=float64)),
             ('rx2_extent', DeviceArray(5.23760933, dtype=float64)),
             ('alcoholP', DeviceArray(95.14091351, dtype=float64)),
             ('etherP', DeviceArray(4.85908649, dtype=float64)),
             ('D1F', DeviceArray(1200.09006814, dtype=float64)),
             ('D1Fx',
              DeviceArray([1.73915272e-05, 8.62596534e-03, 2.17013664e-01,
                           7.74342979e-01], dtype=float64)),
             ('D1Vy',
              DeviceArray([1.66771652e-04, 4.64480150e-02, 7.85744278e-01,
                           1.67640935e-01], dtype=float64)),
             ('D1Lx',
              DeviceArray([1.66771652e-06, 4.64480150e-03, 1.57148856e-01,
                           8.38204675e-01], dtype=float64)),
             ('D1V', DeviceArray(114.29157667, dtype=float64)),
             ('D1L', DeviceArray(1085.79849147, dtype=float64)),
             ('decantW', DeviceArray(14.20150853, dtype=floa

In [19]:
eqs2_wrapped(sol.x)

DeviceArray([ 1.66121067e-14, -3.85561622e-16,  8.53676808e-14,
             -2.00371981e-13,  0.00000000e+00, -6.72205347e-18,
              1.77635684e-15, -8.52651283e-14,  2.27373675e-13,
              1.42108547e-14,  1.77635684e-15, -8.88178420e-16,
             -1.11022302e-16, -5.20417043e-18, -1.66533454e-15,
             -2.44249065e-15, -1.42108547e-14], dtype=float64)